# Data Cleaning Listado de Productores Autorizados

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
# Definición de funciones

def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('- .* -', '', text)  # Remove '- XXXX -'
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252')

Lectura del listado de productores autorizados

In [4]:
listado_productores = load_datasets('../../data/productores_autorizados/')

# 2. Limpieza de los datos

## 2.1 INEGI

In [5]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [6]:
# Revisamos las primeras filas del dataset
dataset_inegi.head()

,MAPA,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2
2,10010096,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",21.883756,-102.357122,1861,F13D18,41,24,17,12
3,10010100,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",21.854599,-102.372745,1879,F13D18,0,0,0,0
4,10010102,NaN,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",21.780180,-102.357295,1861,F13D18,8,*,*,2


In [7]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [8]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT     int64
NOM_ENT    object
CVE_MUN     int64
NOM_MUN    object
dtype: object

In [9]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_clean = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_clean: ", dataset_inegi_clean.shape)

Shape of dataset_inegi:  (299568, 4)
Shape of dataset_inegi_clean:  (2476, 4)


In [10]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_clean.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
0,1,Aguascalientes,1,Aguascalientes
708,1,Aguascalientes,2,Asientos
945,1,Aguascalientes,3,Calvillo
1237,1,Aguascalientes,4,Cosío
1330,1,Aguascalientes,5,Jesús María


In [11]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
299150,32,Zacatecas,54,Villa Hidalgo
299211,32,Zacatecas,55,Villanueva
299363,32,Zacatecas,56,Zacatecas
299484,32,Zacatecas,57,Trancoso
299526,32,Zacatecas,58,Santa María de la Paz


In [12]:
dataset_inegi_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2476 entries, 0 to 299526
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CVE_ENT  2476 non-null   int64 
 1   NOM_ENT  2476 non-null   object
 2   CVE_MUN  2476 non-null   int64 
 3   NOM_MUN  2476 non-null   object
dtypes: int64(2), object(2)
memory usage: 96.7+ KB


In [13]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_clean.nunique())

Los valores únicos en cada columna son:
 CVE_ENT      32
NOM_ENT      32
CVE_MUN     570
NOM_MUN    2332
dtype: int64


In [14]:
# Creamos una columna con la clave única por municipio

dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[
    'CVE_MUN'].astype(str)

dataset_inegi_clean.head()

C:\Users\aorti\AppData\Local\Temp\ipykernel_7452\2388838924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[


,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique
0,1,Aguascalientes,1,Aguascalientes,1-1
708,1,Aguascalientes,2,Asientos,1-2
945,1,Aguascalientes,3,Calvillo,1-3
1237,1,Aguascalientes,4,Cosío,1-4
1330,1,Aguascalientes,5,Jesús María,1-5


In [15]:
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique
299150,32,Zacatecas,54,Villa Hidalgo,32-54
299211,32,Zacatecas,55,Villanueva,32-55
299363,32,Zacatecas,56,Zacatecas,32-56
299484,32,Zacatecas,57,Trancoso,32-57
299526,32,Zacatecas,58,Santa María de la Paz,32-58


In [16]:
dataset_inegi = dataset_inegi.drop_duplicates()

### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [17]:
# Estandarizamos la limpieza de los datos
dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)
dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)

In [18]:
dataset_inegi_clean

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique,NOM_MUN_Clean,NOM_ENT_Clean
0,1,Aguascalientes,1,Aguascalientes,1-1,aguascalientes,aguascalientes
708,1,Aguascalientes,2,Asientos,1-2,asientos,aguascalientes
945,1,Aguascalientes,3,Calvillo,1-3,calvillo,aguascalientes
1237,1,Aguascalientes,4,Cosío,1-4,cosio,aguascalientes
1330,1,Aguascalientes,5,Jesús María,1-5,jesus maria,aguascalientes
...,...,...,...,...,...,...,...
299150,32,Zacatecas,54,Villa Hidalgo,32-54,villa hidalgo,zacatecas
299211,32,Zacatecas,55,Villanueva,32-55,villanueva,zacatecas
299363,32,Zacatecas,56,Zacatecas,32-56,zacatecas,zacatecas
299484,32,Zacatecas,57,Trancoso,32-57,trancoso,zacatecas


## 2.2 Listado de Productores Autorizados

In [19]:
listado_productores.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
       'NOMBRE (S)', 'PAQUETE'],
      dtype='object')

In [20]:
listado_productores.head()

,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2


In [21]:
# Mostramos el tipo de dato de cada columna
listado_productores.dtypes
listado_productores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525720 entries, 0 to 1525719
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   ESTADO            1525720 non-null  object
 1   MUNICIPIO         1450208 non-null  object
 2   ACUSE             1525720 non-null  object
 3   APELLIDO PATERNO  1521426 non-null  object
 4   APELLIDO MATERNO  1497507 non-null  object
 5   NOMBRE (S)        1521427 non-null  object
 6   PAQUETE           1525720 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 81.5+ MB


In [22]:
# Observamos los valores únicos y la cantidad de cada columna, ordenados
COLUMNS_TO_VIEW = ['ESTADO', 'MUNICIPIO', 'PAQUETE']
for column in COLUMNS_TO_VIEW:
    unique_values = listado_productores[column].unique()
    if unique_values.dtype == 'object':
        unique_values = unique_values.astype(str)
        unique_values.sort()
    if len(unique_values) <= 1000:
        print(
            f"\nCantidad de valores unicos {len(unique_values)}. Valores únicos en la columna {column}: {unique_values}")
    else:
        print(f"\nCantidad de valores únicos en {column}: {len(unique_values)}")
        print(f"Valores únicos en la columna {column}:")
        for value in unique_values:
            print(value)


Cantidad de valores unicos 20. Valores únicos en la columna ESTADO: ['AGUASCALIENTES' 'CAMPECHE' 'CHIAPAS' 'CIUDAD DE MEXICO' 'COLIMA'
 'DURANGO' 'GUERRERO' 'HIDALGO' 'JALISCO' 'MEXICO' 'MICHOACAN DE OCAMPO'
 'MORELOS' 'MÉXICO' 'OAXACA' 'PUEBLA' 'QUERETARO DE ARTEAGA'
 'QUINTANA ROO' 'TLAXCALA' 'VERACRUZ DE IGNACIO DE LA LLAVE' 'YUCATAN']

Cantidad de valores únicos en MUNICIPIO: 1695
Valores únicos en la columna MUNICIPIO:
ABALA
ABEJONES
ACACOYAGUA
ACAJETE
ACALA
ACAMBAY
ACANCEH
ACAPETAHUA
ACAPULCO DE JUAREZ
ACATENO
ACATEPEC
ACATIC
ACATLAN
ACATLAN DE JUAREZ
ACATLAN DE PEREZ FIGUEROA
ACATZINGO
ACAXOCHITLAN
ACOLMAN
ACTEOPAN
ACTOPAN
ACUAMANALA DE MIGUEL HIDALGO
ACUITZIO
ACULCO
AGUA BLANCA DE ITURBIDE
AGUASCALIENTES
AGUILILLA
AHUACATLAN
AHUACUOTZINGO
AHUALULCO DE MERCADO
AHUATLAN
AHUAZOTEPEC
AHUEHUETITLA
AJACUBA
AJALPAN
AJUCHITLAN DEL PROGRESO
AKIL
ALBINO ZERTUCHE
ALCOZAUCA DE GUERRERO
ALDAMA
ALFAJAYUCAN
ALJOJUCA
ALMOLOYA
ALMOLOYA DE ALQUISIRAS
ALMOLOYA DE JUAREZ
ALMOLOYA DEL RIO
ALPOYECA

In [23]:
# Filas duplicadas y filas únicas
duplicated_rows_productores = listado_productores.duplicated()
number_of_duplicated_rows_productores = duplicated_rows_productores.sum()
print(f"El número de filas duplicadas es: {number_of_duplicated_rows_productores}")
print(f"El número de filas únicas es: {listado_productores.shape[0] - number_of_duplicated_rows_productores}")

El número de filas duplicadas es: 0
El número de filas únicas es: 1525720


No hay filas duplicadas.

In [24]:
# Valores nulos
listado_productores.isna().sum()

ESTADO                  0
MUNICIPIO           75512
ACUSE                   0
APELLIDO PATERNO     4294
APELLIDO MATERNO    28213
NOMBRE (S)           4293
PAQUETE                 0
dtype: int64

Existen 75512 filas con el municipio nulo. 
Y muchas filas con nombre y apellidos vacíos.


In [25]:
# Analicemos dónde los municipios están vacíos
listado_productores[(listado_productores['MUNICIPIO'].isna())]


,ESTADO,MUNICIPIO,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
1400822,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000001-S000-VZ,ABURTO,ALARCON,ASCENCION,1
1400823,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000002-S000-VZ,ABURTO,ALARCON,GENARO,1
1400824,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000003-S000-VZ,ABURTO,MARTINEZ,MARIA DEL CARMEN,2
1400825,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000004-S000-VZ,ABURTO,MORALES,MIGUEL ANGEL,2
1400826,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-000005-S000-VZ,ALARCON,GUZMAN,MARCIANO,2
...,...,...,...,...,...,...,...
1476329,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081776-S000-VZ,REYES,HERNANDEZ,RUTILIO,1
1476330,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081842-S000-VZ,SANTIAGO,SARMIENTO,LEOBARDO,2
1476331,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081843-S000-VZ,VAZQUEZ,GAONA,JOSE,2
1476332,VERACRUZ DE IGNACIO DE LA LLAVE,NaN,23-PRONAFE-FERT-081845-S000-VZ,VEGA,RODRIGUEZ,JUAN,1


Todas las filas con municipios vacíos vienen del estado de Veracruz de Ignacio de la Llave.

In [26]:
listado_productores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1525720 entries, 0 to 1525719
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   ESTADO            1525720 non-null  object
 1   MUNICIPIO         1450208 non-null  object
 2   ACUSE             1525720 non-null  object
 3   APELLIDO PATERNO  1521426 non-null  object
 4   APELLIDO MATERNO  1497507 non-null  object
 5   NOMBRE (S)        1521427 non-null  object
 6   PAQUETE           1525720 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 81.5+ MB


### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los nombres de municipios en un dataset más ligero llamado Estados_productores.

In [27]:
# Seleccionar solo las dos primeras columnas
Estados_productores = listado_productores[['ESTADO', 'MUNICIPIO']]

# Obtener las filas únicas
Estados_productores = Estados_productores.drop_duplicates()

In [28]:
Estados_productores.shape

(1862, 2)

In [29]:
# Estandarizamos la limpieza de los datos
Estados_productores['ESTADO_Clean'] = Estados_productores['ESTADO'].apply(clean_text)
Estados_productores['MUNICIPIO_Clean'] = Estados_productores['MUNICIPIO'].apply(clean_text)

# 3. Merge de los datasets de INEGI Y PRODUCTORES AUTORIZADOS

A este punto llegamos con 3 datasets:
    1. dataset_inegi_clean: municipios y estados de INEGI con los nombres estandarizados
    2. Productores_Autorizados: listado de productores autorizados con municipios y estados estandarizados
    3. Estados_productores: Municipios y Estados únicos que aparecen en el listado de Productores Autorizados, sin el formato INEGI.

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Estados_productores. Para ello haremos un Left join entre Estados_productores y dataset_inegi_clean.

### 3.1 Left join

In [30]:
# Primero creemos una columna clave en cada dataset -> INEGI

dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)

dataset_inegi_clean["KEY"] = dataset_inegi_clean["NOM_ENT_Clean"] + "-" + dataset_inegi_clean["NOM_MUN_Clean"]

In [31]:
# Primero creemos una columna clave en cada dataset -> Estados productores

Estados_productores["ESTADO_Clean"] = Estados_productores["ESTADO_Clean"].astype(str)
Estados_productores["MUNICIPIO_Clean"] = Estados_productores["MUNICIPIO_Clean"].astype(str)

Estados_productores["KEY"] = Estados_productores["ESTADO_Clean"] + "-" + Estados_productores["MUNICIPIO_Clean"]

In [32]:
dataset_inegi_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2476 entries, 0 to 299526
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CVE_ENT         2476 non-null   int64 
 1   NOM_ENT         2476 non-null   object
 2   CVE_MUN         2476 non-null   int64 
 3   NOM_MUN         2476 non-null   object
 4   CVE_MUN_Unique  2476 non-null   object
 5   NOM_MUN_Clean   2476 non-null   object
 6   NOM_ENT_Clean   2476 non-null   object
 7   KEY             2476 non-null   object
dtypes: int64(2), object(6)
memory usage: 174.1+ KB


In [33]:
Estados_productores.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1862 entries, 0 to 1506712
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ESTADO           1862 non-null   object
 1   MUNICIPIO        1861 non-null   object
 2   ESTADO_Clean     1862 non-null   object
 3   MUNICIPIO_Clean  1862 non-null   object
 4   KEY              1862 non-null   object
dtypes: object(5)
memory usage: 87.3+ KB


In [34]:

# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge(df_inegi, df_prod, key1, key2, threshold=90, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_prod[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_inegi, df_prod, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_inegi', '_prod'))

    return df_merged






In [35]:
# Aplicar la función de coincidencia difusa
diccionario = fuzzy_merge(dataset_inegi_clean, Estados_productores, 'KEY', 'KEY')

# Mostrar el resultado
print(diccionario.head())

   CVE_ENT         NOM_ENT  CVE_MUN         NOM_MUN CVE_MUN_Unique  \
0        1  Aguascalientes        1  Aguascalientes            1-1   
1        1  Aguascalientes        2        Asientos            1-2   
2        1  Aguascalientes        3        Calvillo            1-3   
3        1  Aguascalientes        4           Cosío            1-4   
4        1  Aguascalientes        5     Jesús María            1-5   

    NOM_MUN_Clean   NOM_ENT_Clean                      KEY_inegi  \
0  aguascalientes  aguascalientes  aguascalientes-aguascalientes   
1        asientos  aguascalientes        aguascalientes-asientos   
2        calvillo  aguascalientes        aguascalientes-calvillo   
3           cosio  aguascalientes           aguascalientes-cosio   
4     jesus maria  aguascalientes     aguascalientes-jesus maria   

                      best_match  match_score          ESTADO       MUNICIPIO  \
0  aguascalientes-aguascalientes        100.0  AGUASCALIENTES  AGUASCALIENTES   
1       

In [36]:
diccionario.to_csv('../../data/merged_dataset.csv', index=False)

## 3.4 Productores Autorizados

Esta sección se encarga de completar el listado original de Productores Autorizados con los nombre corregido de INEGI.

In [37]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join
listado_productores['Estado-mun-KEY'] = listado_productores['ESTADO'].astype(str) + '-' + listado_productores[
    'MUNICIPIO'].astype(str)
diccionario['Estado-mun-KEY'] = diccionario['ESTADO'].astype(str) + '-' + diccionario['MUNICIPIO'].astype(str)

# Eliminar los del Estado de Veracruz ya que está dificultando el join.
diccionario_Sin_VC = diccionario[diccionario["NOM_ENT"] != "Veracruz de Ignacio de la Llave"]
print(f"Listado de productores: {listado_productores.shape[0]}")
print(f"Diccionario: {diccionario.shape[0]}")
print(f"Diccionario sin VC: {diccionario_Sin_VC.shape[0]}")


Listado de productores: 1525720
Diccionario: 2600
Diccionario sin VC: 2388


In [38]:
# Revision de duplicados en el diccionario.
#  Revisamos aquellas columnas que tienen valores duplicados
for column in diccionario_Sin_VC.columns:
    # Si hay algún valor duplicado en la columna
    if diccionario_Sin_VC[column].duplicated().any():
        if diccionario_Sin_VC[column].duplicated().sum(): 
            print(f"'{column}' tiene valores duplicados. Tiene {diccionario_Sin_VC[column].duplicated().sum()} valores duplicados.")
    else:
        print(f"columna '{column}' NO tiene valores duplicados.")

# Se supone que no debería haber duplicados en la columna KEY_inegi del diccionario
print(f"El número de filas en el diccionario es: {diccionario_Sin_VC.shape[0]}")
diccionario_Sin_VC = diccionario_Sin_VC.drop_duplicates(subset='KEY_inegi', keep=False)
print(f"El número de filas en el diccionario sin duplicados de KEY_inegi es: {diccionario_Sin_VC.shape[0]}")
# TODO Revisar el diccionario sin duplicados

'CVE_ENT' tiene valores duplicados. Tiene 2357 valores duplicados.
'NOM_ENT' tiene valores duplicados. Tiene 2357 valores duplicados.
'CVE_MUN' tiene valores duplicados. Tiene 1818 valores duplicados.
'NOM_MUN' tiene valores duplicados. Tiene 246 valores duplicados.
'CVE_MUN_Unique' tiene valores duplicados. Tiene 124 valores duplicados.
'NOM_MUN_Clean' tiene valores duplicados. Tiene 246 valores duplicados.
'NOM_ENT_Clean' tiene valores duplicados. Tiene 2357 valores duplicados.
'KEY_inegi' tiene valores duplicados. Tiene 126 valores duplicados.
'best_match' tiene valores duplicados. Tiene 666 valores duplicados.
'match_score' tiene valores duplicados. Tiene 2381 valores duplicados.
'ESTADO' tiene valores duplicados. Tiene 2368 valores duplicados.
'MUNICIPIO' tiene valores duplicados. Tiene 708 valores duplicados.
'ESTADO_Clean' tiene valores duplicados. Tiene 2369 valores duplicados.
'MUNICIPIO_Clean' tiene valores duplicados. Tiene 708 valores duplicados.
'KEY_prod' tiene valores du

In [39]:
# Hacer el join
listado_productores_complete = pd.merge(listado_productores, diccionario_Sin_VC, left_on="Estado-mun-KEY",
                                        right_on="Estado-mun-KEY", how='left', suffixes=('_prod', '_inegi'))

# Codigo para revisar si hay duplicados en el listado de productores. Se puede borrar si es necesario.
# print(f"Columnas de productores completos: {listado_productores_complete.columns}")
# print(f"Columnas de productores: {listado_productores.columns}")
# coinciden = ['MUNICIPIO', 'ACUSE', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
#        'NOMBRE (S)', 'PAQUETE', 'Estado-mun-KEY']
# listado_productores_merged = listado_productores_complete.merge(listado_productores[coinciden], on=['ACUSE', 'Estado-mun-KEY'], how='inner')
# print(listado_productores_merged.shape)
# 
# duplicated_rows = listado_productores_complete[listado_productores_complete.duplicated(['ACUSE','Estado-mun-KEY'], keep=False)]
# print(f"El número de filas en el original es: {listado_productores.shape[0]}")
# print(f"El número de filas en el completo es: {listado_productores_complete.shape[0]}")


In [40]:
print(f"Columnas de productores completos: {listado_productores_complete.info()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538672 entries, 0 to 1538671
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   ESTADO_prod       1538672 non-null  object 
 1   MUNICIPIO_prod    1463160 non-null  object 
 2   ACUSE             1538672 non-null  object 
 3   APELLIDO PATERNO  1534378 non-null  object 
 4   APELLIDO MATERNO  1510226 non-null  object 
 5   NOMBRE (S)        1534379 non-null  object 
 6   PAQUETE           1538672 non-null  int64  
 7   Estado-mun-KEY    1538672 non-null  object 
 8   CVE_ENT           1281548 non-null  float64
 9   NOM_ENT           1281548 non-null  object 
 10  CVE_MUN           1281548 non-null  float64
 11  NOM_MUN           1281548 non-null  object 
 12  CVE_MUN_Unique    1281548 non-null  object 
 13  NOM_MUN_Clean     1281548 non-null  object 
 14  NOM_ENT_Clean     1281548 non-null  object 
 15  KEY_inegi         1281548 non-null  object 
 16  

In [42]:
# Revisamos las columnas del dataset y nos quedamos con las que nos interesan
listado_productores_complete = listado_productores_complete[[
    'ESTADO_prod',
    'MUNICIPIO_prod',
    'ACUSE',
    'APELLIDO PATERNO',
    'APELLIDO MATERNO',
    'NOMBRE (S)',
    'PAQUETE',
    'Estado-mun-KEY',
    'NOM_ENT',
    'NOM_MUN',
    'CVE_ENT',
    'CVE_MUN',
    'CVE_MUN_Unique',  # Añadido
]]

# Separadas las claves de entidad y municipio
# listado_productores_complete[['CVE_ENT', 'CVE_MUN']] = listado_productores_complete['CVE_MUN_Unique'].str.split('-', expand=True)# listado_productores_complete = pd.concat([listado_productores_complete, 
# 


listado_productores_complete.head()

,ESTADO_prod,MUNICIPIO_prod,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE,Estado-mun-KEY,NOM_ENT,NOM_MUN,CVE_ENT,CVE_MUN,CVE_MUN_Unique
0,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000510-S000-AS,AGUILAR,AGUILAR,NICOLAS,2,AGUASCALIENTES-AGUASCALIENTES,Aguascalientes,Aguascalientes,1.0,1.0,1-1
1,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000339-S000-AS,AGUILAR,GARCIA,MANUEL,2,AGUASCALIENTES-AGUASCALIENTES,Aguascalientes,Aguascalientes,1.0,1.0,1-1
2,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000340-S000-AS,AGUILAR,GARCIA,PAULO,2,AGUASCALIENTES-AGUASCALIENTES,Aguascalientes,Aguascalientes,1.0,1.0,1-1
3,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000511-S000-AS,AGUILAR,OROPEZA,MIGUEL,2,AGUASCALIENTES-AGUASCALIENTES,Aguascalientes,Aguascalientes,1.0,1.0,1-1
4,AGUASCALIENTES,AGUASCALIENTES,23-PRONAFE-FERT-000676-S000-AS,ALBA,ESTRADA,MARIA DEL ROSARIO,2,AGUASCALIENTES-AGUASCALIENTES,Aguascalientes,Aguascalientes,1.0,1.0,1-1


In [43]:
listado_productores_complete.tail()


,ESTADO_prod,MUNICIPIO_prod,ACUSE,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE,Estado-mun-KEY,NOM_ENT,NOM_MUN,CVE_ENT,CVE_MUN,CVE_MUN_Unique
1538667,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103370-S000-PL,NaN,NaN,NaN,2,PUEBLA-ZOQUITLAN,Puebla,Zoquitlán,21.0,217.0,21-217
1538668,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103371-S000-PL,NaN,NaN,NaN,2,PUEBLA-ZOQUITLAN,Puebla,Zoquitlán,21.0,217.0,21-217
1538669,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103166-S000-PL,NaN,NaN,NaN,2,PUEBLA-ZOQUITLAN,Puebla,Zoquitlán,21.0,217.0,21-217
1538670,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103373-S000-PL,NaN,NaN,NaN,2,PUEBLA-ZOQUITLAN,Puebla,Zoquitlán,21.0,217.0,21-217
1538671,PUEBLA,ZOQUITLAN,23-PRONAFE-FERT-103374-S000-PL,NaN,NaN,NaN,2,PUEBLA-ZOQUITLAN,Puebla,Zoquitlán,21.0,217.0,21-217


In [49]:
print(f"El numero de filas del merged: {listado_productores_complete.shape[0]}")
print(f"El número de filas en el original es: {listado_productores.shape[0]}")

El numero de filas del merged:1538672
El número de filas en el original es: 1525720


In [45]:
# Descartamos las columnas que no son de interés para el EDA
# listado_productores_complete = listado_productores_complete.drop(['Estado-mun-KEY', 'match_score'], axis=1)

KeyError: "['match_score'] not found in axis"

In [46]:
listado_productores_complete.to_csv('../../data/listado_productores_complete.csv', index=False)